In [20]:
import pandas as pd

df = pd.read_excel('data/信用评分卡模型.xlsx')

df.head(5)

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   月收入     1000 non-null   int64
 1   年龄      1000 non-null   int64
 2   性别      1000 non-null   int64
 3   历史授信额度  1000 non-null   int64
 4   历史违约次数  1000 non-null   int64
 5   信用评分    1000 non-null   int64
dtypes: int64(6)
memory usage: 47.0 KB


In [22]:
df.describe()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,10182.061000,39.075000,0.507000,48783.005000,2.012000,79.558000
std,2719.251125,9.610085,0.500201,27133.636467,1.554436,7.749754
min,5007.000000,20.000000,0.000000,5073.000000,0.000000,60.000000
25%,8160.250000,32.000000,0.000000,26907.750000,1.000000,74.000000
50%,10038.000000,39.000000,1.000000,42948.000000,2.000000,81.000000
75%,12498.250000,47.000000,1.000000,72386.500000,3.000000,86.000000
max,14999.000000,55.000000,1.000000,99991.000000,5.000000,90.000000


In [23]:
df.shape

(1000, 6)

In [24]:
df.isnull().sum()

df.dropna()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73
...,...,...,...,...,...,...
995,12873,52,1,58190,2,87
996,11478,36,0,95688,1,81
997,14105,45,1,76221,2,89
998,11914,37,1,39906,2,83


In [25]:
df.drop_duplicates()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73
...,...,...,...,...,...,...
995,12873,52,1,58190,2,87
996,11478,36,0,95688,1,81
997,14105,45,1,76221,2,89
998,11914,37,1,39906,2,83


In [26]:
X = df.drop('信用评分', axis=1)

y = df['信用评分']



In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

lr = LinearRegression(fit_intercept=True, copy_X=True, n_jobs=None).fit(X_train, y_train)

lr_y_pred = lr.predict(X_test)

df = DecisionTreeRegressor(random_state=0, max_depth=5, min_samples_split=2).fit(X_train, y_train)

df_y_pred = df.predict(X_test)

In [37]:

from sklearn.model_selection import GridSearchCV

lr_param_grid = {'fit_intercept': [True, False], 'copy_X': [True, False]}

df_param_grid = {'max_depth': [3, 5, 7, 9], 'min_samples_split': [2, 3, 4]}

lr_cv = GridSearchCV(estimator=lr, param_grid=lr_param_grid, cv=5)

df_cv = GridSearchCV(estimator=df, param_grid=df_param_grid, cv=5)

lr_cv.fit(X_train, y_train)

df_cv.fit(X_train, y_train)

lr_best_model = lr_cv.best_estimator_

df_best_model = df_cv.best_estimator_

best_lr_y_pred = lr_best_model.predict(X_test)

best_df_y_pred = df_best_model.predict(X_test)



In [38]:
from sklearn.metrics import mean_squared_error, r2_score

mse1 = mean_squared_error(y_test, best_lr_y_pred)

mse2 = mean_squared_error(y_test, best_df_y_pred)

r2_1 = r2_score(y_test, best_lr_y_pred)

r2_2 = r2_score(y_test, best_df_y_pred)

print(mse1, mse2, r2_1, r2_2)

24.375079102050755 22.11553652789338 0.6289381664919083 0.6633351835003942


In [44]:
lr_best_model.coef_

array([ 4.21861078e-04,  1.66227135e-01,  2.57689051e-01,  7.37675240e-05,
       -1.47956252e+00])

In [45]:
df_best_model.feature_importances_

array([0.0020068 , 0.20155199, 0.        , 0.75258667, 0.04385454])

In [47]:
from joblib import dump

dump(df_best_model, 'dt.joblib')

['dt.joblib']